In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Acute_Myeloid_Leukemia"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Acute_Myeloid_Leukemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Step 1: Identify the most relevant directory for Acute Myeloid Leukemia
tcga_subdirs = os.listdir(tcga_root_dir)
print(f"Available TCGA subdirectories: {tcga_subdirs}")

# Look for directories related to Acute Myeloid Leukemia
target_dir = None
for subdir in tcga_subdirs:
    if "Leukemia" in subdir and "Acute" in subdir and "Myeloid" in subdir:
        target_dir = subdir
        break

if target_dir is None:
    print(f"No suitable directory found for {trait}.")
    # Mark the task as completed by creating a JSON record indicating data is not available
    validate_and_save_cohort_info(is_final=False, cohort="TCGA", info_path=json_path, 
                                  is_gene_available=False, is_trait_available=False)
    exit()  # Exit the program

# Step 2: Get file paths for the selected directory
cohort_dir = os.path.join(tcga_root_dir, target_dir)
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

print(f"Selected directory: {target_dir}")
print(f"Clinical data file: {clinical_file_path}")
print(f"Genetic data file: {genetic_file_path}")

# Step 3: Load clinical and genetic data
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')

# Step 4: Print column names of clinical data
print("\nClinical data columns:")
print(clinical_df.columns.tolist())

# Additional basic information
print(f"\nClinical data shape: {clinical_df.shape}")
print(f"Genetic data shape: {genetic_df.shape}")


Available TCGA subdirectories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Canc


Clinical data columns:
['FISH_test_component', 'FISH_test_component_percentage_value', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_LAML', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_LAML', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'acute_myeloid_leukemia_calgb_cytogenetics_risk_category', 'age_at_initial_pathologic_diagnosis', 'atra_exposure', 'cumulative_agent_total_dose', 'cytogenetic_abnormality', 'cytogenetic_abnormality_other', 'cytogenetic_analysis_performed_ind', 'days_to_birth', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'disease_detection_molecular_analysis_method_type', 'fish_evaluation_performed_ind', 'fluorescence_in_situ_hybrid_cytogenetics_metaphase_ncls_rslt_cnt', 'fluorescence_in_situ_hybridization_abnormal_result_indicator', 'form_completion_date', 'gender', 'history_of_neoadjuvant

### Step 2: Find Candidate Demographic Features

In [3]:
# 1. Identify candidate columns for age and gender
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# 2. Extract and preview the candidate columns
clinical_file_path, _ = tcga_get_relevant_filepaths(os.path.join(tcga_root_dir, 'TCGA_Acute_Myeloid_Leukemia_(LAML)'))
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()
print("Age columns preview:")
print(age_preview)

# Extract and preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()
print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [50, 61, 30, 77, 46], 'days_to_birth': [-18385, -22584, -11203, -28124, -16892]}

Gender columns preview:
{'gender': ['MALE', 'FEMALE', 'MALE', 'MALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Select age column - both columns seem to have data but age_at_initial_pathologic_diagnosis is more intuitive
age_col = "age_at_initial_pathologic_diagnosis"

# Select gender column - only one option available and it appears to have consistent data
gender_col = "gender"

# Print the chosen columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize the clinical features
# Get file paths
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Acute_Myeloid_Leukemia_(LAML)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Create standardized clinical features dataframe with trait, age, and gender
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait="AML",  # Using "AML" as the trait name
    age_col=age_col, 
    gender_col=gender_col
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")
print(clinical_features.head())

# Step 2: Normalize gene symbols in gene expression data
# Transpose the genetic data to have genes as rows
genetic_data = genetic_df.copy()

# Normalize gene symbols using the NCBI Gene database synonyms
normalized_gene_data = normalize_gene_symbols_in_index(genetic_data)

# Save normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")
print(f"Normalized gene data shape: {normalized_gene_data.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to get samples as rows, genes as columns
genetic_data_transposed = normalized_gene_data.T

# Ensure clinical and genetic data have the same samples (index values)
common_samples = clinical_features.index.intersection(genetic_data_transposed.index)
clinical_subset = clinical_features.loc[common_samples]
genetic_subset = genetic_data_transposed.loc[common_samples]

# Combine clinical and genetic data
linked_data = pd.concat([clinical_subset, genetic_subset], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values
linked_data = handle_missing_values(linked_data, trait_col="AML")
print(f"After handling missing values - linked data shape: {linked_data.shape}")

# Step 5: Determine biased features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait="AML")
print(f"After removing biased features - linked data shape: {linked_data.shape}")

# Step 6: Validate data quality and save cohort info
# First check if we have both gene and trait data
is_gene_available = linked_data.shape[1] > 3  # More than just AML, Age, Gender
is_trait_available = "AML" in linked_data.columns

# Take notes of special findings
notes = "TCGA AML dataset successfully processed. Contains tumor samples (AML=1) and normal samples (AML=0)."

# Validate the data quality
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=notes
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved due to quality concerns")

Clinical data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/TCGA.csv
Clinical data shape: (200, 3)
                 AML  Age  Gender
sampleID                         
TCGA-AB-2802-03    1   50       1
TCGA-AB-2803-03    1   61       0
TCGA-AB-2804-03    1   30       1
TCGA-AB-2805-03    1   77       1
TCGA-AB-2806-03    1   46       1


Normalized gene data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/TCGA.csv
Normalized gene data shape: (19848, 173)
Linked data shape: (173, 19851)


After handling missing values - linked data shape: (173, 19851)
Quartiles for 'AML':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'AML' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 44.0
  50% (Median): 58.0
  75%: 67.0
Min: 18
Max: 88
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 80 occurrences. This represents 46.24% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

After removing biased features - linked data shape: (173, 19851)
Linked data not saved due to quality concerns
